# HW6 FX Carry (GBP Funding)

In [ ]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.width', 220)
pd.set_option('display.max_columns', 200)

BASE = Path('.').resolve()
OUT = BASE / 'outputs'
FIG = OUT / 'figures'
OUT.mkdir(parents=True, exist_ok=True)
FIG.mkdir(parents=True, exist_ok=True)

print('PWD:', BASE)
for d in [BASE/'data_clean', BASE/'data', BASE/'../../Data']:
    files = sorted(d.glob('*')) if d.exists() else []
    print(f'{d} -> {len(files)} files')
    for f in files[:12]:
        print('  ', f.name)

In [ ]:
# deterministic file selection

def pick_file(paths, patterns):
    tried=[]
    hits=[]
    for d in paths:
        for pat in patterns:
            tried.append((str(d), pat))
            if d.exists():
                hits += list(d.glob(pat))
    hits = sorted(set(hits), key=lambda x: (x.stat().st_mtime, x.name), reverse=True)
    print('patterns tried:', tried)
    print('candidates:', [str(h) for h in hits])
    if not hits:
        return None
    print('chosen:', hits[0])
    return hits[0]


def load_sonia_overnight():
    fp=pick_file([BASE/'data_clean'], ['*IUDSOIA*.parquet','*IUDSOIA*.csv'])
    if fp is None:
        raise FileNotFoundError('No IUDSOIA file found in data_clean')
    if fp.suffix=='.parquet':
        df=pd.read_parquet(fp)
    else:
        df=pd.read_csv(fp)
    if isinstance(df.index, pd.DatetimeIndex):
        s=pd.Series(pd.to_numeric(df.iloc[:,0],errors='coerce').values, index=pd.to_datetime(df.index, errors='coerce'), name='sonia')
    else:
        date_col=next((c for c in df.columns if str(c).upper()=='DATE'), df.columns[0])
        val_col=next((c for c in df.columns if 'IUDSOIA' in str(c).upper()), [c for c in df.columns if c!=date_col][0])
        s=pd.Series(pd.to_numeric(df[val_col],errors='coerce').values, index=pd.to_datetime(df[date_col],errors='coerce'), name='sonia')
    s=s.dropna().sort_index()
    s.index=s.index.tz_localize(None)
    if s.median()>1:
        s=s/100.0
    return s


def load_gbp_5y_funding(sonia_idx):
    fp=pick_file([BASE/'data_clean'], ['*boe*ois*daily*raw*.parquet','*ois*daily*raw*.parquet'])
    if fp is None:
        inv=[x.name for x in sorted((BASE/'data_clean').glob('*'))]
        raise RuntimeError('FAIL: no GBP curve file for 5Y funding. inventory='+str(inv))
    raw=pd.read_parquet(fp)
    req={'__source_file','__sheet','UK OIS spot curve'}
    if not req.issubset(raw.columns):
        raise RuntimeError(f'FAIL: raw OIS columns missing. found={raw.columns.tolist()}')

    sub=raw[raw['__sheet'].astype(str).str.contains('spot curve', case=False, na=False)].copy()
    if sub.empty:
        raise RuntimeError('FAIL: no spot curve rows found for GBP funding 5Y')

    pieces=[]
    for src,g in sub.groupby('__source_file'):
        vals=pd.to_numeric(g['UK OIS spot curve'], errors='coerce').dropna().reset_index(drop=True)
        vals=vals[(vals>-5)&(vals<25)]
        if len(vals)>2 and abs(vals.iloc[0]-1.0)<1e-10 and abs(vals.iloc[1]-1/12)<1e-10:
            vals=vals.iloc[2:].reset_index(drop=True)
        if len(vals)<100:
            continue
        years=[int(x) for x in re.findall(r'(20\d{2})', str(src))]
        if years:
            y0,y1=min(years),max(years)
            idx_pool=pd.DatetimeIndex([d for d in sonia_idx if y0<=d.year<=y1])
        else:
            idx_pool=sonia_idx
        if len(idx_pool)<len(vals):
            idx_pool=sonia_idx[-len(vals):]
        else:
            idx_pool=idx_pool[:len(vals)]
        pieces.append(pd.Series(vals.values, index=idx_pool))

    if not pieces:
        inv=[x.name for x in sorted((BASE/'data_clean').glob('*'))]
        raise RuntimeError('FAIL: unable to construct GBP 5Y funding series. inventory='+str(inv)+' cols='+str(raw.columns.tolist()))

    s=pd.concat(pieces).sort_index()
    s=s[~s.index.duplicated(keep='last')].dropna()
    if s.median()>1:
        s=s/100.0
    s.name='gbp5y_fund'
    return s


def load_fx_usd_per_ccy(currencies):
    fp=pick_file([BASE/'data_clean'], ['*usd_per_ccy*wide*.parquet','*edi*cur*fx_long*.parquet'])
    if fp is None:
        raise FileNotFoundError('No FX file found in data_clean')
    if 'fx_long' in fp.name:
        d=pd.read_parquet(fp)
        fx=d.pivot(index='date', columns='ccy', values='value')
    else:
        fx=pd.read_parquet(fp)
    if not isinstance(fx.index, pd.DatetimeIndex):
        fx.index=pd.to_datetime(fx.index, errors='coerce')
    fx.index=fx.index.tz_localize(None)
    fx.columns=[str(c).upper().strip() for c in fx.columns]
    fx=fx.apply(pd.to_numeric, errors='coerce').sort_index()

    keep=sorted(set(currencies+['GBP']))
    for c in keep:
        if c not in fx.columns:
            fx[c]=np.nan
    fx=fx[keep]

    gbp=fx['GBP'].dropna()
    frac=gbp.between(0.3,5.0).mean() if len(gbp)>0 else 0
    if frac<0.95:
        fx=1.0/fx
        print('FX auto-inverted based on GBP sanity range')
    gbp=fx['GBP'].dropna()
    assert gbp.between(0.3,5.0).mean()>0.95, 'GBP fx quote sanity failed after inversion check'

    jumps=(fx.pct_change().abs()>0.20).sum()
    print('FX abs return >20% counts:')
    print(jumps)
    return fx


def parse_em_curve_csv(fp):
    raw=pd.read_csv(fp)
    out=[]
    cols=list(raw.columns)
    for i in range(0,len(cols),2):
        c0=cols[i]
        c1=cols[i+1] if i+1<len(cols) else None
        if c1 is None:
            continue
        m=re.search(r'GT([A-Z]{3})(\d+)(Y|YR)', str(c0).upper())
        if not m:
            continue
        tmp=pd.DataFrame({
            'date':pd.to_datetime(raw[c0], errors='coerce'),
            'ccy':m.group(1),
            'tenor':float(m.group(2)),
            'rate':pd.to_numeric(raw[c1], errors='coerce')
        }).dropna(subset=['date','rate'])
        out.append(tmp)
    if not out:
        raise RuntimeError('No parsable EM curve columns in '+str(fp))
    return pd.concat(out, ignore_index=True)


def load_em_curves():
    dirs=[BASE/'../../Data', BASE/'data', BASE.parent/'data']
    files=[]
    for d in dirs:
        if d.exists():
            files += list(d.glob('*Emerging*Mkt*YC*.csv'))
    files=sorted(set(files), key=lambda x:(x.stat().st_mtime,x.name), reverse=True)
    if not files:
        raise FileNotFoundError('No EM YC files in ../../Data or ./data')
    print('EM curve files:', [f.name for f in files])
    df=pd.concat([parse_em_curve_csv(f) for f in files], ignore_index=True)
    if df['rate'].median()>1:
        df['rate']=df['rate']/100.0
    df['date']=pd.to_datetime(df['date']).dt.tz_localize(None)
    return df.groupby(['date','ccy','tenor'], as_index=False)['rate'].mean()

EM=['BRL','NGN','PKR','TRY','ZAR']
sonia=load_sonia_overnight()
gbp5=load_gbp_5y_funding(sonia.index)
fx=load_fx_usd_per_ccy(EM)
curves=load_em_curves()

print('coverage sonia',sonia.index.min().date(),sonia.index.max().date(),len(sonia))
print('coverage gbp5 ',gbp5.index.min().date(),gbp5.index.max().date(),len(gbp5))
print('coverage fx   ',fx.index.min().date(),fx.index.max().date(),len(fx))
print('coverage curve',curves.date.min().date(),curves.date.max().date(),len(curves))

In [ ]:
# weekly alignment

def align_weekly(obj, start, end, tol=2):
    target=pd.date_range(start,end,freq='W-WED')
    out=[]; idx=[]
    for d in target:
        cand=[d+pd.Timedelta(days=k) for k in [0,1,2,-1,-2] if abs(k)<=tol]
        c=next((x for x in cand if x in obj.index), None)
        if c is None:
            continue
        out.append(obj.loc[c]); idx.append(d)
    if isinstance(obj,pd.Series):
        return pd.Series(out,index=pd.DatetimeIndex(idx),name=obj.name)
    return pd.DataFrame(out,index=pd.DatetimeIndex(idx),columns=obj.columns)

curve_w=curves.pivot_table(index=['date','ccy'], columns='tenor', values='rate', aggfunc='mean').sort_index()
start=max(sonia.index.min(), gbp5.index.min(), fx.index.min(), curves.date.min())
end=min(sonia.index.max(), gbp5.index.max(), fx.index.max(), curves.date.max())

sonia_w=align_weekly(sonia,start,end)
gbp5_w=align_weekly(gbp5,start,end)
fx_w=align_weekly(fx,start,end)
dates=sonia_w.index.intersection(gbp5_w.index).intersection(fx_w.index)

curve_ccy={}
for c in EM:
    if c not in curve_w.index.get_level_values('ccy'):
        continue
    cdf=align_weekly(curve_w.xs(c,level='ccy'),start,end)
    curve_ccy[c]=cdf.reindex(dates).ffill()

sonia_w=sonia_w.reindex(dates).ffill(); gbp5_w=gbp5_w.reindex(dates).ffill(); fx_w=fx_w.reindex(dates).ffill()

print('weekly n=',len(dates),dates.min().date(),dates.max().date())
print('missing sonia',int(sonia_w.isna().sum()),'gbp5',int(gbp5_w.isna().sum()))
print('missing fx\n',fx_w.isna().sum())
for c in curve_ccy:
    print(c,'curve missing rows',int(curve_ccy[c].isna().any(axis=1).sum()))

In [ ]:
# curve and pricing

def interp_rate(tenors, rates, t):
    x=np.array(tenors,dtype=float); y=np.array(rates,dtype=float)
    m=np.isfinite(x)&np.isfinite(y)
    x=x[m]; y=y[m]
    if len(x)==0:
        return np.nan
    o=np.argsort(x); x=x[o]; y=y[o]
    return float(np.interp(t,x,y,left=y[0],right=y[-1]))

def bootstrap_df_from_par(par_curve, freq=4, max_t=5.0):
    grid=np.arange(1/freq,max_t+1e-12,1/freq)
    dfs={}
    for t in grid:
        s=interp_rate(par_curve.index.values, par_curve.values, t)
        c=s/freq
        pv_prev=sum(c*dfs[pt] for pt in grid if pt<t)
        dfs[t]=max((1-pv_prev)/(1+c),1e-12)
    return pd.Series(dfs)

def price_bond(coupon_rate, par_curve, times, freq=4):
    if len(times)==0:
        return 1.0
    z=bootstrap_df_from_par(par_curve,freq=freq,max_t=max(5.0,float(np.max(times))+0.25))
    df=np.interp(times,z.index.values,z.values,left=z.values[0],right=z.values[-1])
    cf=np.full(len(times),coupon_rate/freq); cf[-1]+=1.0
    return float(np.sum(cf*df))

times_entry=np.arange(0.25,5.0+1e-12,0.25)
dt=1/52
times_exit_full=times_entry-dt
times_exit=times_exit_full[times_exit_full>0]
print('entry head/tail',times_entry[:5],times_entry[-5:])
print('exit  head/tail',times_exit_full[:5],times_exit_full[-5:])
assert np.max(np.abs((times_entry-dt)-times_exit_full))<1e-12

# par sanity
diag=[]
rng=np.random.default_rng(1)
for c in EM:
    if c not in curve_ccy: continue
    cdf=curve_ccy[c].dropna(how='all')
    if cdf.empty: continue
    for k in rng.choice(len(cdf), size=min(4,len(cdf)), replace=False):
        row=cdf.iloc[int(k)].dropna()
        s5=interp_rate(row.index.values,row.values,5.0)
        p0=price_bond(s5,row,times_entry)
        diag.append({'ccy':c,'date':cdf.index[int(k)],'s5':s5,'entry_pv':p0,'dev':p0-1})
par_diag=pd.DataFrame(diag)
print(par_diag.head())
print('entry par pv min/max',par_diag['entry_pv'].min(),par_diag['entry_pv'].max())

In [ ]:
# strategy simulation
rows=[]; plist=[]
for i in range(len(dates)-1):
    t0,t1=dates[i],dates[i+1]
    s5_fund=float(gbp5_w.loc[t0])
    ois=float(sonia_w.loc[t0])
    borrow_rate=ois+0.005
    active=0
    rlist=[]
    spread_list=[]
    for c in EM:
        if c not in curve_ccy: continue
        c0=curve_ccy[c].loc[t0].dropna(); c1=curve_ccy[c].loc[t1].dropna()
        if c0.empty or c1.empty:
            continue
        s5_lend=interp_rate(c0.index.values,c0.values,5.0)
        spread=s5_lend-s5_fund
        trade=bool(np.isfinite(s5_lend) and np.isfinite(s5_fund) and spread>=0.005)

        fx0=fx_w.at[t0,c]; fx1=fx_w.at[t1,c]
        gbp0=fx_w.at[t0,'GBP']; gbp1=fx_w.at[t1,'GBP']
        if not np.isfinite([fx0,fx1,gbp0,gbp1]).all():
            trade=False

        ret=np.nan; pnl=0.0
        if trade:
            units_ccy=10_000_000/fx0
            p1=price_bond(s5_lend,c1,times_exit)
            lend_end=units_ccy*p1*fx1

            debt_units=8_000_000/gbp0
            debt_end=debt_units*(1+borrow_rate/52)*gbp1

            eq0=2_000_000
            eq1=lend_end-debt_end
            pnl=eq1-eq0
            ret=pnl/eq0
            active+=1
            rlist.append(ret)
        spread_list.append(spread)
        rows.append({'date':t0,'next_date':t1,'ccy':c,'active':int(trade),'s5_lend':s5_lend,'s5_fund':s5_fund,'spread':spread,'entry_hurdle':s5_fund+0.005,'ois':ois,'borrow_rate':borrow_rate,'ret':ret,'pnl_usd':pnl})

    plist.append({'date':t0,'port_ret':(float(np.mean(rlist)) if rlist else 0.0),'active_positions':active,'spread_mean':(float(np.mean(spread_list)) if spread_list else np.nan)})

res=pd.DataFrame(rows)
port=pd.DataFrame(plist).set_index('date').sort_index()
port['wealth']=(1+port['port_ret']).cumprod()
port['drawdown']=port['wealth']/port['wealth'].cummax()-1
assert float(port['drawdown'].min()) >= -1-1e-12

active_diag=res.groupby('ccy',as_index=False)['active'].mean().rename(columns={'active':'active_frac'})
avg_active=port['active_positions'].mean()
active_diag.loc[len(active_diag)]={'ccy':'ALL_AVG_ACTIVE_POSITIONS','active_frac':avg_active}
if avg_active>4.9:
    print('WARNING avg active > 4.9; spread diagnostics:')
    print(res.groupby('ccy')['spread'].describe())

active_diag.to_csv(OUT/'active_diagnostics.csv',index=False)
print(active_diag)

In [ ]:
# analytics + artifacts

def ann_sharpe(r):
    s=r.std(ddof=1)
    return np.sqrt(52)*r.mean()/s if s>0 else np.nan

def maxdd(r):
    w=(1+r.fillna(0)).cumprod()
    return float((w/w.cummax()-1).min())

pivot=res.pivot(index='date',columns='ccy',values='ret').sort_index()

stats=[]
for c in pivot.columns:
    rr=pivot[c].dropna()
    if len(rr)==0: continue
    stats.append({'ccy':c,'mean_weekly':rr.mean(),'vol_weekly':rr.std(ddof=1),'ann_sharpe':ann_sharpe(rr),'active_frac':float(res.loc[res.ccy==c,'active'].mean()),'pnl_sum_usd':float(res.loc[res.ccy==c,'pnl_usd'].sum()),'max_dd_wealth':maxdd(rr),'active_weeks':int(rr.notna().sum())})
stats_df=pd.DataFrame(stats).sort_values('ann_sharpe',ascending=False)

corr=pivot.corr(min_periods=25)
base=port['port_ret']; base_sh=ann_sharpe(base); base_dd=float(port['drawdown'].min())

drop=[]
for c in pivot.columns:
    ew=pivot.drop(columns=[c]).mean(axis=1,skipna=True).fillna(0)
    w=(1+ew).cumprod(); dd=float((w/w.cummax()-1).min())
    sh=ann_sharpe(ew)
    drop.append({'ccy_removed':c,'ann_sharpe_drop_one':sh,'max_dd_drop_one':dd,'delta_sharpe':sh-base_sh,'delta_dd':dd-base_dd})
drop_df=pd.DataFrame(drop).sort_values('delta_sharpe')

port_out=port.reset_index()
stats_df.to_csv(OUT/'currency_stats.csv',index=False)
corr.to_csv(OUT/'currency_corr.csv')
drop_df.to_csv(OUT/'drop_one_diagnostic.csv',index=False)
port_out.to_csv(OUT/'portfolio_weekly_returns.csv',index=False)

plt.figure(figsize=(9,4)); plt.plot(port.index,port['wealth']); plt.title('Portfolio wealth'); plt.tight_layout(); plt.savefig(FIG/'portfolio_wealth.png',dpi=130); plt.close()
plt.figure(figsize=(9,4)); plt.plot(port.index,port['drawdown']); plt.title('Portfolio drawdown'); plt.tight_layout(); plt.savefig(FIG/'portfolio_drawdown.png',dpi=130); plt.close()
plt.figure(figsize=(9,4)); plt.plot(port.index,port['active_positions']); plt.title('Active positions per week'); plt.tight_layout(); plt.savefig(FIG/'active_positions.png',dpi=130); plt.close()
plt.figure(figsize=(6,5)); mat=corr.values; plt.imshow(mat,cmap='RdBu_r',vmin=-1,vmax=1); plt.colorbar(); plt.xticks(range(len(corr.columns)),corr.columns,rotation=45,ha='right'); plt.yticks(range(len(corr.index)),corr.index)
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        if np.isfinite(mat[i,j]): plt.text(j,i,f'{mat[i,j]:.2f}',ha='center',va='center',fontsize=8)
plt.tight_layout(); plt.savefig(FIG/'corr_heatmap.png',dpi=130); plt.close()

print('core artifacts written')

In [ ]:
# market factors
hits=[]
for pat in ['*VIX*','*DXY*','*SPX*','*MSCI*','*UST*','*rates*','*factor*']:
    hits += list((BASE/'data_clean').glob(pat))
print('factor hits:', [h.name for h in sorted(set(hits))])

# proxy factors (if no external found)
em_basket=np.log(fx_w[EM]).diff().mean(axis=1)
usd_gbp=-np.log(fx_w['GBP']).diff()
proxy_usd=0.5*usd_gbp + 0.5*em_basket
proxy_rates_ois=sonia_w.diff()
proxy_rates_5y=gbp5_w.diff()

factors=pd.DataFrame({'proxy_usd_broad':proxy_usd,'proxy_em_risk':em_basket,'proxy_d_ois':proxy_rates_ois,'proxy_d_gbp5y':proxy_rates_5y},index=port.index)

mf=pd.concat([port['port_ret'],factors],axis=1).dropna()
mcorr=mf.corr().loc[factors.columns,['port_ret']].rename(columns={'port_ret':'corr_with_port'})
mcorr.to_csv(OUT/'market_factor_corr.csv')

rows=[]
y=mf['port_ret'].values
for fac in factors.columns:
    x=mf[[fac]].values[:,0]
    X=np.column_stack([np.ones(len(x)),x])
    b=np.linalg.lstsq(X,y,rcond=None)[0]
    yh=X@b; e=y-yh
    n=len(y); k=2
    s2=(e@e)/(n-k)
    cov=s2*np.linalg.inv(X.T@X)
    se=np.sqrt(np.diag(cov))
    t=b[1]/se[1] if se[1]>0 else np.nan
    r2=1-(e@e)/np.sum((y-y.mean())**2)
    rows.append({'model':'univariate','factor':fac,'alpha':b[0],'beta':b[1],'t_beta':t,'r2':r2,'n':n})

X=np.column_stack([np.ones(len(mf))]+[mf[c].values for c in factors.columns])
b=np.linalg.lstsq(X,y,rcond=None)[0]
yh=X@b; e=y-yh
n=len(y); k=X.shape[1]
s2=(e@e)/(n-k)
cov=s2*np.linalg.inv(X.T@X)
se=np.sqrt(np.diag(cov)); r2=1-(e@e)/np.sum((y-y.mean())**2)
labels=['const']+list(factors.columns)
for j,lab in enumerate(labels):
    rows.append({'model':'multivariate','factor':lab,'alpha':b[0],'beta':b[j],'t_beta':(b[j]/se[j] if se[j]>0 else np.nan),'r2':r2,'n':n})
reg=pd.DataFrame(rows)
reg.to_csv(OUT/'market_factor_regs.csv',index=False)

for fac in factors.columns:
    tmp=mf[['port_ret',fac]].dropna()
    plt.figure(figsize=(5,4)); plt.scatter(tmp[fac],tmp['port_ret'],s=10,alpha=0.6)
    z=np.polyfit(tmp[fac],tmp['port_ret'],1); xs=np.linspace(tmp[fac].min(),tmp[fac].max(),100)
    plt.plot(xs,z[0]*xs+z[1],color='red'); plt.xlabel(fac); plt.ylabel('port_ret'); plt.tight_layout(); plt.savefig(FIG/f'factor_scatter_{fac}.png',dpi=120); plt.close()

print(mcorr)
print(reg.head(10))

In [ ]:
# write markdown report from outputs
stats=pd.read_csv(OUT/'currency_stats.csv')
corr=pd.read_csv(OUT/'currency_corr.csv',index_col=0)
drop=pd.read_csv(OUT/'drop_one_diagnostic.csv')
port_csv=pd.read_csv(OUT/'portfolio_weekly_returns.csv',parse_dates=['date'])
mc=pd.read_csv(OUT/'market_factor_corr.csv',index_col=0)
mr=pd.read_csv(OUT/'market_factor_regs.csv')
active=pd.read_csv(OUT/'active_diagnostics.csv')


def md_table(df, nd=6):
    cols=list(df.columns)
    lines=['| '+' | '.join(cols)+' |','|'+'|'.join(['---']*len(cols))+'|']
    for _,r in df.iterrows():
        vals=[]
        for c in cols:
            v=r[c]
            if isinstance(v,float): vals.append(f'{v:.{nd}f}')
            else: vals.append(str(v))
        lines.append('| '+' | '.join(vals)+' |')
    return '\\n'.join(lines)

r=port_csv['port_ret']
ann_ret=r.mean()*52
ann_vol=r.std(ddof=1)*np.sqrt(52)
ann_sh=ann_ret/ann_vol if ann_vol>0 else np.nan

text=[]
text.append('# HW6 FX Carry Report')
text.append('## 1) Spec recap')
text.append('- Weekly strategy; lend $10MM in EM 5Y par bond, fund $8MM in GBP at OIS+50bp, equity $2MM.')
text.append('- Entry filter uses true GBP 5Y funding series: trade if s5_lend >= s5_fund + 50bp.')
text.append('- MTM uses explicit quarterly schedule shifted by 1/52 at exit; no separate accrual added to lending leg.')
text.append('- USD accounting for all PnL and portfolio aggregation.')
text.append('## 2) Data & coverage')
text.append(f"- Weekly sample: {port_csv['date'].min().date()} to {port_csv['date'].max().date()}, N={len(port_csv)}.")
text.append('- Inputs loaded from ./data_clean/ and curves from ../../Data/ or ./data fallback.')
text.append('## 3) Methodology')
text.append('- Curve pricing uses linear interpolation + recursive discount bootstrap from par rates (Zero/Spot style).')
text.append('- Cashflow shift validation shown in notebook output (entry and exit times).')
text.append('- GBP 5Y funding built from local BoE OIS spot-curve archive (not overnight proxy).')
text.append('## 4) Results')
text.append(md_table(stats))
text.append('| metric | value |')
text.append('|---|---:|')
text.append(f'| ann return | {ann_ret:.6f} |')
text.append(f'| ann vol | {ann_vol:.6f} |')
text.append(f'| ann sharpe | {ann_sh:.6f} |')
text.append(f"| max drawdown | {port_csv['drawdown'].min():.6f} |")
text.append('![wealth](outputs/figures/portfolio_wealth.png)')
text.append('![drawdown](outputs/figures/portfolio_drawdown.png)')
text.append('![active](outputs/figures/active_positions.png)')
text.append('![corr](outputs/figures/corr_heatmap.png)')
text.append('## 5) Correlation and drop-one')
text.append(md_table(corr.reset_index().rename(columns={corr.index.name or 'index':'ccy'}), nd=3))
text.append(md_table(drop))
text.append('## 6) Market Factors')
text.append('- No external factor files found; used labeled proxy factors.')
text.append(md_table(mc.reset_index()))
text.append(md_table(mr))
text.append('Carry/crash interpretation: FX/risk proxies explain part of variation; carry can be dominated by adverse FX shocks in risk-off episodes.')
text.append('## 7) Robustness & limitations')
text.append('- FX inversion checks, >20% move flags, weekly missingness diagnostics, par PV sanity, and drawdown assertion included.')
text.append('- Limitation: available local GBP curve data is reconstructed from BoE raw archive format and may not provide full pillar metadata.')
text.append('')

(BASE/'hw6_fx_carry_report.md').write_text('\n\n'.join(text))
print('report written to', BASE/'hw6_fx_carry_report.md')